# Example #2 - Working with files

Let's make sure our data files are there. The `os` library will be very helpful.

In [ ]:
from os import listdir
from os.path import join

listdir('.')

In [ ]:
listdir('../data')

We will rely on `pandas`'s built in `read_csv` function. 

In [ ]:
import pandas as pd
pd.read_csv?

What other "read" functions does `pandas` have?

In [ ]:
import re
regex = re.compile(r'read')
list(filter(regex.match, dir(pd)))

Let's focus on `read_csv` for now, and investigate some data from [openflights.org](openflights.org)

In [ ]:
routes = pd.read_csv(join('..', 'data', 'routes.csv'))
airports = pd.read_csv(join('..', 'data', 'airports.csv'))

In [ ]:
routes.head()

In [ ]:
routes.info()

Looks like there's some missing data points...can we visualize what that looks like?

In [ ]:
import missingno as msno

`missingno` is a library that wraps together handy data utilities to get a visual summary of data completeness. We will only scratch the surface of this library.

In [ ]:
%matplotlib inline
msno.matrix(routes)

This gives a quick display to visually check out your data. The sparkline on the right shows the completeness of the data. Let's check out airports.

In [ ]:
airports.info()

In [ ]:
msno.matrix(airports)

Cool! Overall the csvs look pretty clean, with relatively complete data. We've got ~68000 airline routes and ~7000 airports around the world. Let's see if this list is real by trying to find everyone's favorite airport - LaGuardia (LGA).

In [ ]:
airports[airports['IATA'] == 'LGA']

Awesome. How about finding some other things like the highest airport in the world, highest airport in the country that comes last alphabetically, and the highest airport in Australia.

In [ ]:
highest = airports.sort_values('altitude', ascending=False).iloc[0]
highestInLastAlpha = airports.sort_values(['country', 'altitude'], ascending=[False, False]).iloc[0]
highestInAustralia = airports[airports['country'] == 'Australia'].sort_values('altitude', ascending=False).iloc[0]

In [ ]:
print(f'Highest airport: \n{highest}\n\n')
print(f'Highest airport in the country that comes last alphabetically: \n{highestInLastAlpha}\n\n')
print(f'Highest airport in Australia: \n{highestInAustralia}\n\n')

Let's do some more practical things with this data like finding the 10 busiest routes in the world.

In [ ]:
busiest = routes.groupby(['source', 'dest']).count()['airline_id'].nlargest(10)
busiest

We can merge in the names of the airports.

In [ ]:
busiest = pd.DataFrame(busiest).reset_index()
busiest.head()

In [ ]:
for sd in ['source', 'dest']:
    busiest = busiest.merge(airports[['IATA', 'name']].set_index('IATA'), how='left', left_on=sd, right_on='IATA')

busiest